In [4]:
from dataclasses import dataclass
from typing import Dict, Optional, Tuple, Union
from stable_baselines3.common.type_aliases import GymStepReturn

import gymnasium as gym
import numpy as np
import pytest
from gymnasium import spaces
from gymnasium.spaces.space import Space
from torch import nn


from stable_baselines3 import SAC, A2C, PPO
from stable_baselines3.common.env_util import make_atari_env
from stable_baselines3.common.vec_env import VecFrameStack
# from stable_baselines3.sac.custom_policies import Cus, CustomContinousCritic, CustomSACPolicy
from stable_baselines3.ppo.custom_policies import CustomActorCriticPolicy
# from stable_baselines3 import 
import torch

from stable_baselines3.common.env_checker import check_env

ImportError: cannot import name 'CustomACPolicy' from 'stable_baselines3.ppo.policies' (/home/irobotics/stable-baselines3/stable_baselines3/ppo/policies.py)

In [ ]:
class FakeImageEnv(gym.Env):
    """
    Fake image environment for testing purposes, it mimics Atari games.

    :param action_dim: Number of discrete actions
    :param screen_height: Height of the image
    :param screen_width: Width of the image
    :param n_channels: Number of color channels
    :param discrete: Create discrete action space instead of continuous
    :param channel_first: Put channels on first axis instead of last
    """

    def __init__(
        self,
        action_dim: int = 6,
        screen_height: int = 64,
        screen_width: int = 64,
        n_channels: int = 3,
        discrete: bool = False,
        channel_first: bool = False,
    ) -> None:
        self.observation_shape = (screen_height, screen_width, n_channels)
        if channel_first:
            self.observation_shape = (n_channels, screen_height, screen_width)
        self.observation_space = spaces.Box(low=0, high=255, shape=self.observation_shape, dtype=np.uint8)
        if discrete:
            self.action_space = spaces.Discrete(action_dim)
        else:
            self.action_space = spaces.Box(low=-1, high=1, shape=(action_dim,), dtype=np.float32)
        self.ep_length = 10
        self.current_step = 0

    def reset(self, *, seed: Optional[int] = None, options: Optional[Dict] = None) -> Tuple[np.ndarray, Dict]:
        if seed is not None:
            super().reset(seed=seed)
        self.current_step = 0
        return self.observation_space.sample(), {}

    def step(self, action: Union[np.ndarray, int]) -> GymStepReturn:
        reward = 0.0
        self.current_step += 1
        terminated = False
        truncated = self.current_step >= self.ep_length
        return self.observation_space.sample(), reward, terminated, truncated, {}

    def render(self, mode: str = "human") -> None:
        pass

In [5]:
inp = torch.randn(1,3,64,64).to("cuda")


In [9]:
env = FakeImageEnv(discrete=False)
policy_kwargs = dict(
    features_extractor_class=CustomFeaturesExtractor,
    features_extractor_kwargs=dict(features_dim=128),
)
model = PPO(CustomActorCriticPolicy, env, verbose=1, policy_kwargs=policy_kwargs)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


/home/irobotics/stable-baselines3/sb3-dev/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/irobotics/stable-baselines3/sb3-dev/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [10]:
check_env(env, warn=True)

In [5]:
class somePolicy(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
                    nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, padding=1),
                    nn.ReLU(),
                    nn.Conv2d(in_channels=64, out_channels=32, kernel_size=3, padding=1), #64*64*64
                    nn.ReLU(),
                    nn.MaxPool2d(kernel_size=2, stride=2),
                    nn.Conv2d(in_channels=32, out_channels=16, kernel_size=3, padding=1), #32*32*32
                    nn.ReLU(),
                    nn.Conv2d(in_channels=16, out_channels=8, kernel_size=3, padding=1), #16*32*32
                    nn.MaxPool2d(kernel_size=2, stride=2),
                    nn.ReLU(),
                    nn.Flatten(),
                    nn.Linear(8 * 16 * 16, 6) # 8*16*16
                )
    def forward(self, x):
            x /= 255.
            return self.net(x)

In [6]:
sp = somePolicy().to(device="cuda")

In [11]:
model.learn(total_timesteps=1000)

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 10       |
|    ep_rew_mean     | 0        |
| time/              |          |
|    fps             | 440      |
|    iterations      | 1        |
|    time_elapsed    | 4        |
|    total_timesteps | 2048     |
---------------------------------


KeyboardInterrupt: 

: 